# Scripts Antoine

In [1]:
# Parameters
file_location = "images/20 mins_t9.jpg"
particle_pixel_size = 8
volume_fraction = 0.45

## Data Processing

In [2]:
import numpy as np
import trackpy as tp
import pandas as pd
from skimage.exposure import equalize_adapthist
import functions as fun

# Opening, preprocessing the image
img, img_info = fun.load_img(file_location)
img_array = np.array(img)
img_norm = equalize_adapthist(img_array)

# Particle detection
locations_comp = tp.locate(img_norm, 9, minmass=4, separation=9, invert=True)
locations = tp.locate(img_norm, 9, minmass=1, separation=5, invert=True, threshold=0.05)

# Particle pair computation
rho_ctrs, g_exp = fun.particle_pair(locations, img_info, particle_pixel_size, rhos=np.linspace(0.01,5*particle_pixel_size, 100))

# Percus Yevick comparison
g_PY = fun.percus_yevick(rho_ctrs, volume_fraction)

fun.export_g_rho(rho_ctrs, g_exp, g_PY)
print(' > {:05d} particles found'.format(len(locations)))

 > 02373 particles found


## Plotting figure with particle positions

In [3]:
from bokeh.plotting import figure, show

f = figure(width=1000, height=1000, tooltips=[('x', '$x'),('y', '$y'),('lum', '@image')])
f.image([1-img_norm], x=0, y=0, dw=img_info['width'], dh=img_info['height'])
f.scatter(locations_comp['x'], locations_comp['y'], marker='+', line_color='lime', size=5)
f.scatter(locations['x'], locations['y'], marker='x', line_color='red', size=5)
show(f)

## Plotting figure with $g(\rho)$

In [6]:
from bokeh.plotting import figure, show

f = figure()
f.line(rho_ctrs, g_PY, line_color='darkred')
f.scatter(rho_ctrs, g_exp, marker='x', size=6)
f.line([1,1], [0,np.max(g_exp)], line_dash='dotted', line_color='black')
f.line(rho_ctrs, np.ones(np.size(rho_ctrs)), line_dash='dashed', line_color='black')
show(f)

## 'Mass' of particles

In [7]:
from bokeh.plotting import figure, show

mss, bins = np.histogram(locations['mass'], bins=100)
f = figure()
f.quad(left=bins[:-1], right=bins[1:], top=mss, bottom=0*mss)
show(f)